In [0]:
import os
import nbformat
from nbconvert import PythonExporter, HTMLExporter
from nbconvert.preprocessors import CellExecutionError, ExecutePreprocessor
from traitlets.config import Config
import warnings

# Install openml since it's not in colab by default
!pip install --quiet openml
warnings.filterwarnings('ignore') 

In [6]:
# Convert notebook to python
from google.colab import drive
drive.mount('/content/drive')

assignment_path = '/content/drive/My Drive/assignment-3-nickvdw/Assignment 3.ipynb'
export_path = '/content/drive/My Drive/assignment-3-nickvdw/solution.py'
template_path = '/content/drive/My Drive/assignment-3-nickvdw/submit/Template.ipynb'
submission_path = '/content/drive/My Drive/assignment-3-nickvdw/Submission.html'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
# Check if we have all the required files
!pwd
%cd /content/drive/My Drive/assignment-3-nickvdw/
!ls

/content/drive/My Drive/assignment-3-nickvdw
/content/drive/My Drive/assignment-3-nickvdw
'Assignment 3.ipynb'   model_2_2.h5	   test_image_one.png
 model_1_1.h5	       model_2_2.p	   test_output_embedding.p
 model_1_1.p	       model_3_3_test.h5   toy_example.h5
 model_1_2.h5	       model_3_3_test.p    toy_example.p
 model_1_2.p	       model_4_1.h5	   train_output_embedding.p
 model_1_3.h5	       model_4_1.p	   val_output_embedding.p
 model_1_3.p	       __pycache__	   verify_google_colab.ipynb
 model_2_1.h5	       solution.py	   verify.py
 model_2_1.p	       submit


In [8]:
print("Converting: {0}".format(assignment_path))
exporter = PythonExporter()
# source is a tuple of python source code, meta contains metadata
(source, meta) = exporter.from_filename(assignment_path)
with open(export_path, 'w+') as fh:
    fh.writelines(source)
    fh.writelines("last_edit = '{}'".format(meta['metadata']['modified_date']))

# Run solution notebook
with open(template_path) as f:
    snb = nbformat.read(f, as_version=4)
ep = ExecutePreprocessor(timeout=2000, kernel_name='python3')

try:
    print("Running notebook... (may take a while)")
    out = ep.preprocess(snb, {'metadata': {'path': './'}})
except CellExecutionError:
    out = None
    msg = 'Error executing the notebook "%s".\n\n' % template_path
    msg += 'See notebook "%s" for the traceback.' % template_path
    print(msg)
    raise
finally:
    # Save notebook
    with open(template_path, mode='w', encoding='utf-8') as f:
        nbformat.write(snb, f)

# Export as HTML (PDF is too much hassle)
print("All good. Building report.")
c = Config()
c.TagRemovePreprocessor.enabled=True
c.TagRemovePreprocessor.remove_input_tags = set(["hide_input"])
c.preprocessors = ["TagRemovePreprocessor"]

html_exporter = HTMLExporter(config=c)
html_data, resources = html_exporter.from_notebook_node(snb)
html_data = html_data.replace('</head>', '<style>pre{font-family: "Times New Roman", Times, serif;}</style></head>')

with open(submission_path, "wb") as f:
    f.write(html_data.encode('utf8'))
    f.close()
#Cleanup
#os.remove("solution.py")
print("Done.")

Converting: /content/drive/My Drive/assignment-3-nickvdw/Assignment 3.ipynb
Running notebook... (may take a while)
All good. Building report.
Done.
